In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
tokenizer = Tokenizer()

data = 'In the town of Athy one Jeremy Lanigan \n Battered away'
corpus = data.lower().split('\n')

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

In [ ]:
input_sequences = []

for line in corpus: 
  # texts_to_sequences returns a list of lists, grab the first one 
  token_list = tokenizer.texts_to_sequences([line])[0]
  
  # loop through sentence creating n-gram sentences out of the tokens 
  # ex: 
  #   [4 2]
  #   [4 2 66]
  #   [4 2 55 8]
  #   ...
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

In [ ]:
# find the longest sentence in the corpus 
max_sequence_len = max([len(x) for x in input_sequences])

In [ ]:
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [ ]:
print(input_sequences)

[[ 0  0  0  0  0  0  1  2]
 [ 0  0  0  0  0  1  2  3]
 [ 0  0  0  0  1  2  3  4]
 [ 0  0  0  1  2  3  4  5]
 [ 0  0  1  2  3  4  5  6]
 [ 0  1  2  3  4  5  6  7]
 [ 1  2  3  4  5  6  7  8]
 [ 0  0  0  0  0  0  9 10]]


In [ ]:
# now take everything but the last value of each sequence as the X and the last as the y
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

print(X)
print(y)

[[0 0 0 0 0 0 1]
 [0 0 0 0 0 1 2]
 [0 0 0 0 1 2 3]
 [0 0 0 1 2 3 4]
 [0 0 1 2 3 4 5]
 [0 1 2 3 4 5 6]
 [1 2 3 4 5 6 7]
 [0 0 0 0 0 0 9]]
[ 2  3  4  5  6  7  8 10]


In [ ]:
# one-hot encode the labels 
# treat this as classification
# number of classes is the total number of words 
# this means that the y would be an array of length=total_words
# where the token value is used as the index and is set to 1
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [ ]:
model = tf.keras.models.Sequential()
# subtract one from the max_sequence_len because we use the last element as label
model.add(tf.keras.layers.Embedding(total_words, 64, input_length=max_sequence_len - 1))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20)))
model.add(tf.keras.layers.Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=500, verbose=1)

Epoch 1/500
1/1 [==============================] - 5s 5s/step - loss: 2.3936 - accuracy: 0.1250
Epoch 2/500
1/1 [==============================] - 0s 17ms/step - loss: 2.3879 - accuracy: 0.1250
Epoch 3/500
1/1 [==============================] - 0s 15ms/step - loss: 2.3821 - accuracy: 0.3750
Epoch 4/500
1/1 [==============================] - 0s 23ms/step - loss: 2.3763 - accuracy: 0.3750
Epoch 5/500
1/1 [==============================] - 0s 18ms/step - loss: 2.3705 - accuracy: 0.3750
Epoch 6/500
1/1 [==============================] - 0s 20ms/step - loss: 2.3647 - accuracy: 0.3750
Epoch 7/500
1/1 [==============================] - 0s 18ms/step - loss: 2.3587 - accuracy: 0.3750
Epoch 8/500
1/1 [==============================] - 0s 21ms/step - loss: 2.3526 - accuracy: 0.3750
Epoch 9/500
1/1 [==============================] - 0s 14ms/step - loss: 2.3464 - accuracy: 0.3750
Epoch 10/500
1/1 [==============================] - 0s 19ms/step - loss: 2.3400 - accuracy: 0.2500
Epoch 11/500
1/1 [===

In [ ]:
# predicting sentence
seed_text = 'Laurence went to Dublin'

token_list = tokenizer.texts_to_sequences([seed_text])[0]

In [ ]:
token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')

In [ ]:
predicted = model.predict(token_list)
predicted = np.argmax(predicted, axis=-1)[0]

1/1 [==============================] - 1s 845ms/step


In [ ]:
output_word = tokenizer.index_word[predicted]
print(f'{seed_text} {output_word}')

Laurence went to Dublin away


In [ ]:
# predict 10 words ahead
for _ in range(10):
  token_list = tokenizer.texts_to_sequences([seed_text])[0]
  token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
  predicted = model.predict(token_list, verbose=0)
  predicted = np.argmax(predicted, axis=-1)[0]
  output_word = tokenizer.index_word[predicted]
  seed_text += f' {output_word}'

print(seed_text)

Laurence went to Dublin away away away the town athy one jeremy lanigan lanigan


In [ ]:
# model for a larger corpus 
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(total_words, 100, input_length=max_sequence_len-1),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150)),
    tf.keras.layers.Dense(total_words, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.01), metrcis=['accuracy'])
history = model.fit(X, y, epochs=100, verbose=1)